In [5]:
import numpy as np
from netCDF4 import Dataset

# Define the variables and times
variables = ['lma', 'chl', 'lwc']
times = range(13)

def process_file(variable, time_index):
    # Define file paths
    input_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/fitting/fitted_prescribed_lai/transposed_shift_fittings_lwc_time_{time_index:02d}.nc"
    output_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/{variable}_aviris_dangermond_clima_fit_time_{time_index:02d}.nc"
    #reference_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/index/{variable}_aviris_dangermond_time_{time_index:02d}.nc"
    reference_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/index/chl_aviris_dangermond_time_{time_index:02d}.nc"


    mask_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/index/chl_aviris_dangermond_time_{time_index:02d}.nc"

    # Read the input data
    with Dataset(input_file, 'r') as src:
        lat_data = src.variables['lat'][:]
        lon_data = src.variables['lon'][:]
        var_data = src.variables[variable][:]

    # Read the reference file to get the time variable and its attributes
    with Dataset(reference_file, 'r') as ref:
        time_data = ref.variables['time'][:]
        time_units = ref.variables['time'].units
        time_calendar = ref.variables['time'].calendar


    # Read the mask data
    with Dataset(mask_file, 'r') as mask_src:
        chl_data = mask_src.variables['chl'][:]

    # Create the mask based on NaN values in CHL data
    mask = np.isnan(chl_data)

    # Apply the mask operation
    var_data = np.where(mask, np.nan, var_data)

    # Reshape var_data to match the dimensions (time, latitude, longitude)
    time_dim = 1
    lat_dim = len(lat_data)
    lon_dim = len(lon_data)
    var_reshaped = var_data.reshape((time_dim, lat_dim, lon_dim))

    # Create the new NetCDF file and define dimensions
    with Dataset(output_file, 'w', format='NETCDF4') as dst:
        dst.createDimension('time', time_dim)
        dst.createDimension('latitude', lat_dim)
        dst.createDimension('longitude', lon_dim)

        # Create time variable
        time = dst.createVariable('time', 'f8', ('time',), fill_value=np.nan)
        time[:] = time_data
        time.units = time_units
        time.calendar = time_calendar

        # Create longitude variable
        longitude = dst.createVariable('longitude', 'f8', ('longitude',), fill_value=np.nan)
        longitude[:] = lon_data
        longitude.standard_name = "longitude"
        longitude.long_name = "longitude"
        longitude.units = "degrees_east"
        longitude.axis = "X"

        # Create latitude variable
        latitude = dst.createVariable('latitude', 'f8', ('latitude',), fill_value=np.nan)
        latitude[:] = lat_data
        latitude.standard_name = "latitude"
        latitude.long_name = "latitude"
        latitude.units = "degrees_north"
        latitude.axis = "Y"

        # Create variable
        var = dst.createVariable(variable, 'f4', ('time', 'latitude', 'longitude',), fill_value=np.nan)
        var[:] = var_reshaped
        var.crs = "EPSG:32610"

    print(f"{variable.upper()} data for time {time_index:02d} successfully extracted and written to {output_file}")

# Loop through each variable and time, and process the corresponding files
for variable in variables:
    for time_index in times:
        process_file(variable, time_index)


LMA data for time 00 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_00.nc
LMA data for time 01 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_01.nc
LMA data for time 02 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_02.nc
LMA data for time 03 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_03.nc
LMA data for time 04 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma

In [8]:
import numpy as np
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime

# Define the variables and times
variables = ['lma', 'lwc', 'chl']
times = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# Given dates
dates = ["2022-02-24T00:00:00.000000", "2022-02-28T00:00:00.000000", "2022-03-08T00:00:00.000000",
         "2022-03-16T00:00:00.000000", "2022-03-22T00:00:00.000000", "2022-04-05T00:00:00.000000",
         "2022-04-12T00:00:00.000000", "2022-04-20T00:00:00.000000", "2022-04-29T00:00:00.000000",
         "2022-05-03T00:00:00.000000", "2022-05-11T00:00:00.000000", "2022-05-17T00:00:00.000000",
         "2022-05-29T00:00:00.000000"]

def process_files_for_variable(variable):
    # Define file path for output
    output_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/{variable}_aviris_dangermond_clima_fit.nc"

    # Initialize the output NetCDF file
    with Dataset(output_file, 'w', format='NETCDF4') as dst:
        dst.createDimension('time', len(times))
        dst.createDimension('latitude', 3200)
        dst.createDimension('longitude', 3200)

        time_var = dst.createVariable('time', 'f8', ('time',))
        time_var.units = "seconds since 2000-01-01 00:00:00"
        time_var.calendar = "standard"
        time_var[:] = date2num([datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%f") for date in dates], units=time_var.units, calendar=time_var.calendar)

        latitude = dst.createVariable('latitude', 'f8', ('latitude',), fill_value=np.nan)
        latitude.standard_name = "latitude"
        latitude.long_name = "latitude"
        latitude.units = "degrees_north"
        latitude.axis = "Y"

        longitude = dst.createVariable('longitude', 'f8', ('longitude',), fill_value=np.nan)
        longitude.standard_name = "longitude"
        longitude.long_name = "longitude"
        longitude.units = "degrees_east"
        longitude.axis = "X"

        var = dst.createVariable(variable, 'f4', ('time', 'latitude', 'longitude',), fill_value=np.nan)
        var.crs = "EPSG:32610"

        for i, time_index in enumerate(times):
            # Define file paths for input and mask
            input_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/fitting/fitted_prescribed_lai/transposed_shift_fittings_lwc_time_{time_index}.nc"
            mask_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/index/chl_aviris_dangermond_time_{time_index}.nc"

            # Read the input data
            with Dataset(input_file, 'r') as src:
                lat_data = src.variables['lat'][:]
                lon_data = src.variables['lon'][:]
                var_data = src.variables[variable][:].astype(np.float32)

            # Read the mask data
            with Dataset(mask_file, 'r') as mask_src:
                chl_data = mask_src.variables['chl'][:]

            # Create the mask based on NaN values in CHL data
            mask = np.isnan(chl_data)

            # Apply the mask operation
            var_data = np.where(mask, np.nan, var_data)

            # Write the data to the output file
            if i == 0:
                latitude[:] = lat_data
                longitude[:] = lon_data

            var[i, :, :] = var_data

            print(f"{variable.upper()} data for time {time_index} successfully extracted and written to {output_file}")

# Loop through each variable and process the corresponding files
for variable in variables:
    process_files_for_variable(variable)


LMA data for time 00 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit.nc
LMA data for time 01 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit.nc
LMA data for time 02 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit.nc
LMA data for time 03 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit.nc
LMA data for time 04 successfully extracted and written to /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit.nc


In [6]:
import subprocess
import os

# Define the variables and times
variables = ['lma', 'chl', 'lwc']
times = range(13)

# Path to the remap grid file
remap_grid_file = "../California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc"

def regrid_file(variable, time_index):
    input_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/{variable}_aviris_dangermond_clima_fit_time_{time_index:02d}.nc"
    output_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/{variable}_aviris_dangermond_clima_fit_time_{time_index:02d}_reg.nc"
    
    # Construct the cdo command
    cdo_command = [
        'cdo', f'remapbil,{remap_grid_file}', 
        input_file, 
        output_file
    ]
    
    # Execute the cdo command
    subprocess.run(cdo_command, check=True)
    print(f"Regridded file created: {output_file}")

# Loop through each variable and time, and regrid the corresponding files
for variable in variables:
    for time_index in times:
        regrid_file(variable, time_index)


cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3873696)
Regridded file created: /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_00_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3870389)
Regridded file created: /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_01_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3867850)
Regridded file created: /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_time_02_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3865656)
Regridded file created: /

In [9]:
import subprocess

# Define the variables
variables = ['lma', 'lwc', 'chl']

# Path to the remap grid file
remap_grid_file = "/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc"

def regrid_file(variable):
    input_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/{variable}_aviris_dangermond_clima_fit.nc"
    output_file = f"/net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/{variable}_aviris_dangermond_clima_fit_reg.nc"
    
    # Construct the cdo command
    cdo_command = [
        'cdo', f'remapbil,{remap_grid_file}', 
        input_file, 
        output_file
    ]
    
    try:
        # Execute the cdo command and capture the output
        result = subprocess.run(cdo_command, check=True, capture_output=True, text=True)
        print(f"Regridded file created: {output_file}")
    except subprocess.CalledProcessError as e:
        # Print the error message
        print(f"Error occurred while regridding {variable}:")
        print(e.stderr)

# Loop through each variable and regrid the corresponding files
for variable in variables:
    regrid_file(variable)


Regridded file created: /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lma_aviris_dangermond_clima_fit_reg.nc
Regridded file created: /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/lwc_aviris_dangermond_clima_fit_reg.nc
Regridded file created: /net/fluo/data3/data/FluoData1/students/renato/aviris_dangermond/traits/datasets/clima_fit_prescribed_lai/chl_aviris_dangermond_clima_fit_reg.nc
